In [29]:
import arcpy
import os
import sys
from pathlib import Path

from arcgis.features import GeoAccessor

sys.path.append('../../src')

from ba_data_paths import ba_data

In [16]:
data = Path(os.path.abspath('../../data'))

raw = data/'raw'
raw_gdb = raw/'raw.gdb'

interim = data/'interim'
interim_gdb = interim/'interim.gdb'

cbsa = raw_gdb/'sea_cbsa'

arcpy.env.overwriteOutput = True

In [21]:
lyr_bg = ba_data.layer_block_group
arcpy.management.GetCount(lyr_bg)

<Result '217195'>

In [22]:
lyr_cbsa = arcpy.management.MakeFeatureLayer(str(cbsa))[0]
lyr_cbsa

In [23]:
arcpy.management.SelectLayerByLocation(lyr_bg, overlap_type='HAVE_THEIR_CENTER_IN', select_features=lyr_cbsa)

<Result 'GPLYR_{A2DD9CA8-8B55-42BD-9707-DAB5627BAFA3}'>

In [24]:
arcpy.management.GetCount(lyr_bg)

<Result '2476'>

In [25]:
bg = arcpy.management.CopyFeatures(lyr_bg, str(raw_gdb/'sea_block_group'))[0]

In [126]:
bg_df = GeoAccessor.from_featureclass(bg)

In [127]:
bg_df.spatial.project(4326)

True

In [129]:
bg_df.spatial.to_featureclass(bg)

'D:\\projects\\geoai-retail\\data\\raw\\raw.gdb\\sea_block_group'

In [26]:
arcpy.management.GetCount(bg)

<Result '2476'>

In [27]:
ace = interim_gdb/'business_AceHardware'

In [31]:
ace_df = GeoAccessor.from_featureclass(str(ace))
ace_df.head()

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,SHAPE
0,1,174244442,ACE HARDWARE,TOLT AVE,CARNATION,WA,Washington,98014,,44413005,...,3.0,1,,2,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13571448.928367466, ""y"": 6048542.661566..."
1,2,401684669,ACE HARDWARE,N PEARL ST,TACOMA,WA,Washington,98407,2416,45322015,...,23.0,,,8,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13638162.699199878, ""y"": 5986847.462380..."
2,3,414508330,ACE HARDWARE,NE SUNSET BLVD,RENTON,WA,Washington,98059,4005,44413005,...,12.0,1,,6,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13598566.356324706, ""y"": 6024780.675253..."
3,4,421027779,ACE HARDWARE,PO BOX 73486,PUYALLUP,WA,Washington,98373,0486,44413005,...,6.0,,,3,Postal,M,100.0,INFOGROUP,2,"{""x"": -13619961.962455178, ""y"": 5966870.621299..."
4,5,425477834,ACE HARDWARE,4TH AVE,SEATTLE,WA,Washington,98101,1106,44413005,...,12.0,1,,5,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13618759.711954605, ""y"": 6042612.547375..."


In [36]:
ace_df.groupby(['NAICS', 'SIC']).count()

,,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,SALESVOL,...,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,SHAPE
NAICS,SIC,,,,,,,,,,,,,,,,,,,,,
44413005,525104,11,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
45322015,594710,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [48]:
biz_str = r'D:\arcgis\ba_data\Data\Business Data\BA_BUS_2018.gdb\us_businesses'
lyr_biz = arcpy.management.MakeFeatureLayer(biz_str)[0]
arcpy.management.SelectLayerByLocation(lyr_biz, overlap_type='WITHIN', select_features=lyr_cbsa)

In [49]:
naics_codes = ['444130', '444110']
sql = ' OR '.join([f"NAICS LIKE '{code}%'" for code in naics_codes])
sql

"NAICS LIKE '444130%' OR NAICS LIKE '444110%'"

In [50]:
arcpy.management.SelectLayerByAttribute(lyr_biz, selection_type='SUBSET_SELECTION', where_clause=sql)
arcpy.management.GetCount(lyr_biz)

<Result '344'>

In [51]:
biz_fc = arcpy.management.CopyFeatures(lyr_biz, str(raw_gdb/'sea_hardware_home_improvement'))[0]
arcpy.management.GetCount(biz_fc)

<Result '344'>

In [62]:
biz_df = GeoAccessor.from_featureclass(biz_fc)
biz_df.spatial.project(4326)
biz_df.head()

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,SHAPE
0,1,371889957,GRAHAM ACE HARDWARE,224TH ST E,GRAHAM,WA,Washington,98338,5704,44413005,...,15.0,1,,6,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.29909999999998, ""y"": 47.05410000000..."
1,2,460555345,CAPITOL LUMBER & DOOR,MOUNTAIN HWY E,SPANAWAY,WA,Washington,98387,9452,44411001,...,25.0,,,7,StreetAddress,M,100.0,INFOGROUP,0,"{""x"": -122.36899999999999, ""y"": 47.01870000000..."
2,3,502088164,BUILDERS FIRST SOURCE,208TH ST E,SPANAWAY,WA,Washington,98387,8185,44411001,...,40.0,,,7,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.41119999999995, ""y"": 47.06890000000..."
3,4,718164872,RANDLES SURE GROW TOPSOIL,CANYON RD E,PUYALLUP,WA,Washington,98375,2310,44411001,...,11.0,,,6,StreetAddress,M,100.0,INFOGROUP,0,"{""x"": -122.35839999999997, ""y"": 47.08280000000..."
4,5,805003878,NORTHERN ACOUSTICAL SYSTS CO,,GRAHAM,WA,Washington,98338,,44411001,...,5.0,,,4,Postal,M,100.0,INFOGROUP,1,"{""x"": -122.29459999999992, ""y"": 47.07650000000..."


In [68]:
biz_ace_df = biz_df[biz_df.CONAME.str.contains('ACE HARDWARE')].copy()
biz_ace_df.drop('OBJECTID', axis=1, inplace=True)
biz_ace_df.head()

,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,SIC,...,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,SHAPE
0,371889957,GRAHAM ACE HARDWARE,224TH ST E,GRAHAM,WA,Washington,98338,5704,44413005,525104,...,15.0,1,,6,PointAddress,M,100.0000,INFOGROUP,0,"{""x"": -122.29909999999998, ""y"": 47.05410000000..."
8,460556608,OAKBROOK ACE HARDWARE,STEILACOOM BLVD SW,TACOMA,WA,Washington,98498,6154,44413005,525104,...,9.0,1,,3,StreetAddress,M,96.3594,INFOGROUP,0,"{""x"": -122.54490000000003, ""y"": 47.18000000000..."
15,405129289,GIG HARBOR ACE HARDWARE,POINT FOSDICK DR NW,GIG HARBOR,WA,Washington,98335,1711,44413005,525104,...,38.0,1,,7,StreetAddress,M,100.0000,INFOGROUP,0,"{""x"": -122.57929999999996, ""y"": 47.30130000000..."
33,216082099,SOUTH END ACE HARDWARE,PACIFIC AVE S,SPANAWAY,WA,Washington,98387,8395,44413005,525104,...,16.0,1,,5,PointAddress,M,100.0000,INFOGROUP,0,"{""x"": -122.43459999999999, ""y"": 47.08740000000..."
39,721714069,ACE HARDWARE,112TH ST S,PARKLAND,WA,Washington,98444,5711,44413005,525104,...,8.0,,,3,PointAddress,M,95.4063,INFOGROUP,0,"{""x"": -122.43199999999993, ""y"": 47.15610000000..."


In [69]:
biz_not_ace_df = biz_df[~biz_df.CONAME.str.contains('ACE HARDWARE')]
biz_not_ace_df.drop('OBJECTID', axis=1, inplace=True)
biz_not_ace_df.head()

,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,SIC,...,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,SHAPE
1,460555345,CAPITOL LUMBER & DOOR,MOUNTAIN HWY E,SPANAWAY,WA,Washington,98387,9452,44411001,521126,...,25.0,,,7,StreetAddress,M,100.0,INFOGROUP,0,"{""x"": -122.36899999999999, ""y"": 47.01870000000..."
2,502088164,BUILDERS FIRST SOURCE,208TH ST E,SPANAWAY,WA,Washington,98387,8185,44411001,521126,...,40.0,,,7,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.41119999999995, ""y"": 47.06890000000..."
3,718164872,RANDLES SURE GROW TOPSOIL,CANYON RD E,PUYALLUP,WA,Washington,98375,2310,44411001,521126,...,11.0,,,6,StreetAddress,M,100.0,INFOGROUP,0,"{""x"": -122.35839999999997, ""y"": 47.08280000000..."
4,805003878,NORTHERN ACOUSTICAL SYSTS CO,,GRAHAM,WA,Washington,98338,,44411001,521126,...,5.0,,,4,Postal,M,100.0,INFOGROUP,1,"{""x"": -122.29459999999992, ""y"": 47.07650000000..."
5,216099309,LOWE'S HOME IMPROVEMENT,100TH ST SW,LAKEWOOD,WA,Washington,98499,3863,44411002,521138,...,100.0,L,,8,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -122.50489999999998, ""y"": 47.16760000000..."


In [70]:
biz_ace_df.spatial.to_featureclass(str(raw_gdb/'sea_ace'))

'D:\\projects\\geoai-retail\\data\\raw\\raw.gdb\\sea_ace'

In [71]:
biz_not_ace_df.spatial.to_featureclass(str(raw_gdb/'sea_ace_comp'))

'D:\\projects\\geoai-retail\\data\\raw\\raw.gdb\\sea_ace_comp'

In [73]:
lyr_ace = arcpy.management.MakeFeatureLayer('D:\\projects\\geoai-retail\\data\\raw\\raw.gdb\\sea_ace')[0]
lyr_ace

In [120]:
arcpy.management.SelectLayerByLocation(lyr_biz, overlap_type='WITHIN_A_DISTANCE', select_features=lyr_ace, search_distance='0.1 miles')

<Result 'GPLYR_{30770628-D4C5-41A8-A185-DF97821F4AE7}'>

In [122]:
near_df = GeoAccessor.from_featureclass(lyr_biz)
near_df = near_df[~near_df['CONAME'].str.contains('ACE HARDWARE')].copy()
near_df.head()

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,...,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,SHAPE
0,1044235,221903354,DOMINO'S,224TH ST E,GRAHAM,WA,Washington,98338,5728,72251115,...,30.0,D,,3,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13614340.328170117, ""y"": 5950811.010389..."
1,1456971,262888209,VCA PACIFIC AVENUE SOUTH,PACIFIC AVE S,SPANAWAY,WA,Washington,98387,8395,54194002,...,7.0,ERS1,,3,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13629357.32747813, ""y"": 5956237.2189951..."
2,1457059,262899545,AMICI ITALIAN EATERY,224TH ST E,GRAHAM,WA,Washington,98338,5704,72251117,...,25.0,,I,6,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13614273.53647564, ""y"": 5950909.0444182..."
3,1715288,356705962,FITNESS 19,224TH ST E,GRAHAM,WA,Washington,98338,5704,71394011,...,14.0,9,,7,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13614273.53647564, ""y"": 5950909.0444182..."
5,2299402,401464729,GRAHAM NAILS,224TH ST E,GRAHAM,WA,Washington,98338,7086,81211202,...,2.0,,,2,PointAddress,M,100.0,INFOGROUP,0,"{""x"": -13614217.87673024, ""y"": 5950811.0103892..."


In [131]:
fc_near_coname = near_df.spatial.to_featureclass('memory/near_coname')
fc_near_coname

'memory\\near_coname'

In [150]:
biz_ace_fc = str(raw_gdb/'sea_ace')
id_fld = 'LOCNUM'

near_coname_tbl = arcpy.analysis.Near(biz_ace_fc, fc_near_coname, method='GEODESIC')
near_coname_df = GeoAccessor.from_featureclass(near_coname_tbl)
near_coname_df = near_coname_df[[id_fld, 'NEAR_FID']].copy()
near_df = GeoAccessor.from_featureclass(fc_near_coname).set_index('OBJECTID')[['LOCNUM', 'CONAME', 'NAICS']].copy()
near_coname_df = near_coname_df.join(near_df, on='NEAR_FID', rsuffix="_NEAR").drop('NEAR_FID', axis=1)
near_coname_df.columns = ['LOCNUM', 'near_id', 'near_name', 'near_naics']
near_coname_df

,LOCNUM,near_id,near_name,near_naics
0,371889957,262899545,AMICI ITALIAN EATERY,72251117
1,460556608,714960080,OAKBROOK ACE,44413005
2,405129289,310755749,UPS STORE,54186007
3,216082099,371688334,WILSON MOTORS,44111001
4,721714069,708101138,MITCHELL HARDWARE PARKLAND INC,44413005
5,421027779,214354987,TRUE LIGHT ELECTRIC INC,23821007
6,677129595,427271359,BEYOND THE BOTTLE,52239001
7,688834225,634767172,SIMPLY PAWS LLC,45391003
8,427271369,174148577,AGRISHOP LAWN & TRACTOR,44422012
9,715565781,390553485,ISLAND MARKET FLORAL,44511003


In [124]:
naics_df = near_df.groupby('NAICS').count().sort_values('OBJECTID', ascending=False)[['OBJECTID']].copy().reset_index()
naics_df.columns = ['NAICS', 'count']
naics_df[naics_df['count'] > 1]
naics_lookup = GeoAccessor.from_table(r'D:\arcgis\ba_data\Data\Business Data\biz.gdb\NAICS').set_index('NAICS', drop=True).drop('OBJECTID', axis=1)
naics_df.join(naics_lookup, on='NAICS')

,NAICS,count,NAICSDESC
0,72251117,64,RESTAURANTS
1,99999004,52,NONCLASSIFIED ESTABLISHMENTS
2,81211202,34,BEAUTY SALONS
3,52211001,23,AUTOMATED TELLER MACHINES
4,54111002,18,ATTORNEYS
5,53121003,17,REAL ESTATE
6,52421001,14,INSURANCE
7,72251115,13,PIZZA
8,44511003,12,GROCERS-RETAIL
9,23822025,11,PLUMBING CONTRACTORS
